# Testing - Imitation
In this code, we aim to demonstrate how accurately the RDR model created by the proxy replicates the performance of the original model. By evaluating the accuracy, we can assess whether the explanations provided by the RDR model effectively represent the behavior of the original model.

In [1]:
import sys
sys.path.append('../src/')
from rdr import RDR

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

pd.options.display.max_columns = None
pd.options.display.max_rows = None

## Import Model Libraries

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.neural_network import MLPClassifier
from pytorch_tabnet.tab_model import TabNetClassifier

## 1. Load dataset

### 1.1. Load the dataset and store it in a variable named **df**

In [4]:
df = pd.read_csv('../data/star_classification.csv')
display(df.head())
display(df.info())

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   obj_ID       100000 non-null  float64
 1   alpha        100000 non-null  float64
 2   delta        100000 non-null  float64
 3   u            100000 non-null  float64
 4   g            100000 non-null  float64
 5   r            100000 non-null  float64
 6   i            100000 non-null  float64
 7   z            100000 non-null  float64
 8   run_ID       100000 non-null  int64  
 9   rerun_ID     100000 non-null  int64  
 10  cam_col      100000 non-null  int64  
 11  field_ID     100000 non-null  int64  
 12  spec_obj_ID  100000 non-null  float64
 13  class        100000 non-null  object 
 14  redshift     100000 non-null  float64
 15  plate        100000 non-null  int64  
 16  MJD          100000 non-null  int64  
 17  fiber_ID     100000 non-null  int64  
dtypes: float64(10), int64(7),

None

### 1.2. Define and train Label Encoder

In [5]:
le = LabelEncoder()
le.fit(df['class'])

LabelEncoder()

## 2. Testing RDR model's imitation accuracy
At this phase, the RDR model will be created and compared against the original models. The conventional models to be used consists of:

**White box Model**
- Decision Tree Classifier

**Black box Model**
- Random Forest
- Support Vector Machine
- XGBoost
- MLP
- TabNet

Furthermore, testing will be carried out on data samples which differs from 25%, 50%, 75%, and 100% (its original size).

### 2.1. Testing with sample data size : 25%

In [6]:
def remove_outliers(df, threshold=2.5):
    z_scores = np.abs(stats.zscore(df.select_dtypes(include='number')))
    outliers = (z_scores > threshold).any(axis=1)
    return df[~outliers]

# Sample size : 25%
df25 = df.sample(frac=0.25, random_state=42)

# remove outlier
df25 = remove_outliers(df25)

# define features and label
label = df25['class']
features = df25.drop(
            columns=['obj_ID', 'run_ID', 'rerun_ID', 'field_ID', 'spec_obj_ID','fiber_ID','class'],
            axis=1)

# split data for training and testing model
# train : test = 80 : 20
X25_train, X25_test, y25_train, y25_test = train_test_split(features,
                                                            label,
                                                            test_size=0.2,
                                                            random_state=42)

#### 2.1.1. RDR against Decision Tree Classifier

In [7]:
# CREATE AND TRAIN DECISION TREE CLASSIFIER
dtf25 = DecisionTreeClassifier()
dtf25.fit(X25_train, y25_train)

DecisionTreeClassifier()

In [8]:
# CREATE PREDICTION DATASET - USING ALL DATA IN DATASET
prediction_dataset = dtf25.predict(features)

In [9]:
# CREATE AND TRAIN RDR MODEL
rdr = RDR(
        antecedent='GALAXY',
        categorical_attr=['cam_col','plate'],
        comp_operator='>=')

dtf_rdr25 = rdr.fit(features, pd.DataFrame(prediction_dataset))

# PREDICT USING DECISION TREE AND RDR
dtf25_pred = dtf25.predict(X25_test)
dtf_rdr25_pred = dtf_rdr25.predict(X25_test)

#### 2.1.2. RDR against Random Forest Classifier

In [10]:
# CREATE AND TRAIN RANDOM FOREST CLASSIFIER
rf25 = RandomForestClassifier()
rf25.fit(X25_train, y25_train)

RandomForestClassifier()

In [11]:
# CREATE PREDICTION DATASET - USING ALL DATA IN DATASET
prediction_dataset = rf25.predict(features)

In [12]:
# CREATE AND TRAIN RDR MODEL
rdr = RDR(
        antecedent='GALAXY',
        categorical_attr=['cam_col','plate'],
        comp_operator='>=')

rf_rdr25 = rdr.fit(features, pd.DataFrame(prediction_dataset))

# PREDICT USING DECISION TREE AND RDR
rf25_pred = rf25.predict(X25_test)
rf_rdr25_pred = rf_rdr25.predict(X25_test)

#### 2.1.3. RDR against Support Vector Machine

In [13]:
param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['rbf','linear' 'poly', 'sigmoid']}

svm = SVC()

grid = GridSearchCV(svm,param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid.fit(X25_train, y25_train)

print("Best parameters found: ", grid.best_params_)
print("Best cross-validation accuracy: {:.2f}".format(grid.best_score_))

c:\Users\Vieri\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
80 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Vieri\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Vieri\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "c:\Users\Vieri\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py

Best parameters found:  {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
Best cross-validation accuracy: 0.79


In [22]:
# CREATE AND TRAIN SUPPORT VECTOR MACHINE
svm25 = SVC(C= 100, gamma= 0.001, kernel= 'rbf')
svm25.fit(X25_train, y25_train)

SVC(C=100, gamma=0.001)

In [23]:
# CREATE PREDICTION DATASET - USING ALL DATA IN DATASET
prediction_dataset = svm25.predict(features)

In [24]:
# CREATE AND TRAIN RDR MODEL
rdr = RDR(
        antecedent='GALAXY',
        categorical_attr=['cam_col','plate'],
        comp_operator='>=')

svm_rdr25 = rdr.fit(features, pd.DataFrame(prediction_dataset))

# PREDICT USING DECISION TREE AND RDR
svm25_pred = svm25.predict(X25_test)
svm_rdr25_pred = svm_rdr25.predict(X25_test)

#### 2.1.4. RDR against XGBoost Classifier

In [25]:
# CREATE AND TRAIN XGBOOST CLASSIFIER
train_dmatrix = xgb.DMatrix(data=X25_train, label=le.transform(y25_train))
xgboost = xgb.train({}, train_dmatrix)

In [26]:
# CREATE PREDICTION DATASET - USING ALL DATA IN DATASET
dmatrix = xgb.DMatrix(
            data=features,
            label=le.transform(label))

xgb25_pred = xgboost.predict(dmatrix)
prediction_dataset = le.inverse_transform(np.round(xgb25_pred).astype(int))

In [27]:
# CREATE AND TRAIN RDR MODEL
rdr = RDR(
        antecedent='GALAXY',
        categorical_attr=['cam_col','plate'],
        comp_operator='>=')

xgb_rdr25 = rdr.fit(features, pd.DataFrame(prediction_dataset))

# PREDICT USING DECISION TREE AND RDR
dmatrix = xgb.DMatrix(
            data=X25_test,
            label=le.transform(y25_test))

xgb25_pred = xgboost.predict(dmatrix)
xgb25_pred = le.inverse_transform(np.round(xgb25_pred).astype(int))

xgb_rdr25_pred = xgb_rdr25.predict(X25_test)

#### 2.1.5. RDR against Multi-layer Perceptron (MLP) Classifier

In [14]:
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'learning_rate': ['constant', 'adaptive'],
    'learning_rate_init': [0.1, 0.01, 0.001],
    'batch_size': [32, 64],
    'max_iter': [300, 500],
    'alpha': [0.0001, 0.001],
    'activation': ['relu', 'tanh']
}

mlp = MLPClassifier()

grid_search = GridSearchCV(mlp, param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(X25_train, y25_train)

print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation accuracy: {:.2f}".format(grid_search.best_score_))

Best parameters found:  {'activation': 'relu', 'alpha': 0.0001, 'batch_size': 64, 'hidden_layer_sizes': (50, 50), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 300}
Best cross-validation accuracy: 0.70


In [28]:
# CREATE AND TRAIN MLP CLASSIFIER
mlp25 = MLPClassifier(
    max_iter=300,
    batch_size=64,
    hidden_layer_sizes=(50, 50),
    learning_rate='constant',
    learning_rate_init=0.001,
    alpha=0.0001,
    activation='relu'
)
mlp25.fit(X25_train, y25_train)

MLPClassifier(batch_size=64, hidden_layer_sizes=(50, 50), max_iter=300)

In [29]:
# CREATE PREDICTION DATASET - USING ALL DATA IN DATASET
prediction_dataset = mlp25.predict(features)

In [30]:
# CREATE AND TRAIN RDR MODEL
rdr = RDR(
        antecedent='GALAXY',
        categorical_attr=['cam_col','plate'],
        comp_operator='>=')

mlp_rdr25 = rdr.fit(features, pd.DataFrame(prediction_dataset))

# PREDICT - MLP AND RDR
mlp25_pred = mlp25.predict(X25_test)
mlp_rdr25_pred = mlp_rdr25.predict(X25_test)

#### 2.1.6. RDR against TabNet Classifier

In [19]:
# CREATE AND TRAIN TabNet Classifier
tabnet25 = TabNetClassifier()
tabnet25.fit(X25_train.values, y25_train, max_epochs=100)

c:\Users\Vieri\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
c:\Users\Vieri\AppData\Local\Programs\Python\Python312\Lib\site-packages\pytorch_tabnet\abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


epoch 0  | loss: 0.8756  |  0:00:00s
epoch 1  | loss: 0.41183 |  0:00:01s
epoch 2  | loss: 0.27131 |  0:00:02s
epoch 3  | loss: 0.18768 |  0:00:03s
epoch 4  | loss: 0.14744 |  0:00:04s
epoch 5  | loss: 0.14364 |  0:00:05s
epoch 6  | loss: 0.1317  |  0:00:06s
epoch 7  | loss: 0.1358  |  0:00:07s
epoch 8  | loss: 0.12568 |  0:00:08s
epoch 9  | loss: 0.12121 |  0:00:09s
epoch 10 | loss: 0.12134 |  0:00:10s
epoch 11 | loss: 0.11721 |  0:00:10s
epoch 12 | loss: 0.1118  |  0:00:11s
epoch 13 | loss: 0.12513 |  0:00:12s
epoch 14 | loss: 0.11757 |  0:00:13s
epoch 15 | loss: 0.11734 |  0:00:14s
epoch 16 | loss: 0.12211 |  0:00:15s
epoch 17 | loss: 0.11791 |  0:00:16s
epoch 18 | loss: 0.11041 |  0:00:17s
epoch 19 | loss: 0.11168 |  0:00:18s
epoch 20 | loss: 0.11308 |  0:00:19s
epoch 21 | loss: 0.11612 |  0:00:20s
epoch 22 | loss: 0.10839 |  0:00:21s
epoch 23 | loss: 0.12207 |  0:00:21s
epoch 24 | loss: 0.11813 |  0:00:22s
epoch 25 | loss: 0.11486 |  0:00:23s
epoch 26 | loss: 0.11413 |  0:00:24s
e

In [20]:
# CREATE PREDICTION DATASET - USING ALL DATA IN DATASET
prediction_dataset = tabnet25.predict(features.values)

In [21]:
# CREATE AND TRAIN RDR MODEL
rdr = RDR(
        antecedent='GALAXY',
        categorical_attr=['cam_col','plate'],
        comp_operator='>=')

tabnet_rdr25 = rdr.fit(features, pd.DataFrame(prediction_dataset))

# PREDICT - TabNet AND RDR
tabnet25_pred = tabnet25.predict(X25_test.values)
tabnet_rdr25_pred = tabnet_rdr25.predict(X25_test)

## 5. Accuracy Comparison between RDR model and conventional models
To streamline the process, the accuracy comparison for each sample size will be presented by invoking the `display_accuracy` function.

In [32]:
def display_accuracy(
        y_test,
        dtf_pred, dtf_rdr_pred,
        rf_pred, rf_rdr_pred,
        xgb_pred, xgb_rdr_pred,
        svm_pred, svm_rdr_pred,
        mlp_pred, mlp_rdr_pred,
        tabnet_pred, tabnet_rdr_pred
    ) -> None:

    accuracy_table = []

    accuracy_table.append([
        "Decision Tree",
        metrics.accuracy_score(y_test,dtf_pred),
        metrics.accuracy_score(y_test, dtf_rdr_pred),
        metrics.accuracy_score(dtf_pred, dtf_rdr_pred)
    ])

    accuracy_table.append([
        "Random Forest",
        metrics.accuracy_score(y_test,rf_pred),
        metrics.accuracy_score(y_test, rf_rdr_pred),
        metrics.accuracy_score(rf_pred, rf_rdr_pred)
    ])

    accuracy_table.append([
        "XGBoost",
        metrics.accuracy_score(y_test,xgb_pred),
        metrics.accuracy_score(y_test, xgb_rdr_pred),
        metrics.accuracy_score(xgb_pred, xgb_rdr_pred)
    ])

    accuracy_table.append([
        "Support Vector Machine",
        metrics.accuracy_score(y_test,svm_pred),
        metrics.accuracy_score(y_test, svm_rdr_pred),
        metrics.accuracy_score(svm_pred, svm_rdr_pred)
    ])

    accuracy_table.append([
        "Multi-layer Perceptron",
        metrics.accuracy_score(y_test,mlp_pred),
        metrics.accuracy_score(y_test, mlp_rdr_pred),
        metrics.accuracy_score(mlp_pred, mlp_rdr_pred)
    ])

    accuracy_table.append([
        "TabNet",
        metrics.accuracy_score(y_test,tabnet_pred),
        metrics.accuracy_score(y_test, tabnet_rdr_pred),
        metrics.accuracy_score(tabnet_pred, tabnet_rdr_pred)
    ])

    accuracy_table = pd.DataFrame(accuracy_table, columns=["Model", "Accuracy", "RDR Accuracy", "Difference"])
    pd.set_option('display.precision', 15)

    display(accuracy_table)

    return

In [33]:
print("Accuracy comparison for 25% sample size:")
display_accuracy(
    y25_test,
    dtf25_pred, dtf_rdr25_pred,
    rf25_pred, rf_rdr25_pred,
    xgb25_pred, xgb_rdr25_pred,
    svm25_pred, svm_rdr25_pred,
    mlp25_pred, mlp_rdr25_pred,
    tabnet25_pred, tabnet_rdr25_pred
)

Accuracy comparison for 25% sample size:


,Model,Accuracy,RDR Accuracy,Difference
0,Decision Tree,0.958769633507853,0.863656195462478,0.888307155322862
1,Random Forest,0.972513089005236,0.895724258289703,0.910558464223386
2,XGBoost,0.965968586387435,0.836823734729494,0.857547993019197
3,Support Vector Machine,0.802574171029668,0.765270506108202,0.849258289703316
4,Multi-layer Perceptron,0.629799301919721,0.629581151832461,0.975785340314136
5,TabNet,0.968150087260035,0.886780104712042,0.908595113438045
